In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import json
import numpy as np
from sklearn.inspection import permutation_importance
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# Function to load and update puzzle data
def load_puzzle_data(file_path):
    !cd buggle-training-data && git pull
    with open('buggle-training-data/training_data.json', 'r') as f:
      puzzle_data = json.load(f)
      return puzzle_data

def extract_features(puzzle_data):
    all_features = []
    for puzzle in puzzle_data:
        try:
            matrix = puzzle['features']['matrix']
            if any(len(row) != 5 for row in matrix) or len(matrix) != 5:  # Ensure each matrix is 5x5
                print("Error: Matrix is not 5x5")
                continue  # Skip this puzzle

            ascii_matrix = np.array([[ord(char) for char in row] for row in matrix])
            all_features.append(ascii_matrix)

        except KeyError as e:
            print(f"Missing key in puzzle: {e}")
            continue
        except Exception as e:
            print(f"Error processing puzzle: {e}")
            continue

    return np.array(all_features)

def preprocess_data(all_features, outcomes):
    # Existing preprocessing steps...
    all_features = np.expand_dims(all_features, axis=-1)
    original_shape = all_features.shape
    all_features = all_features.reshape(-1, original_shape[1] * original_shape[2])
    scaler = MinMaxScaler()
    all_features = scaler.fit_transform(all_features)
    all_features = all_features.reshape(original_shape[0], original_shape[1], original_shape[2], 1)

    # Updated splitting logic to include a validation set
    X_train, X_temp, y_train, y_temp = train_test_split(all_features, outcomes, test_size=0.3, random_state=42)  # 70% train, 30% temp
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # Split temp into 50% val, 50% test

    return X_train, X_val, X_test, y_train, y_val, y_test

def build_model():
    model = Sequential([
      Input(shape=(5, 5, 1)),  # Define the input shape explicitly here
      Conv2D(32, (3, 3), activation='relu', padding='same'),
      MaxPooling2D((2, 2)),
      Dropout(0.25),
      Conv2D(64, (3, 3), activation='relu', padding='same'),
      MaxPooling2D((2, 2)),
      Dropout(0.25),
      Flatten(),
      Dense(128, activation='relu'),
      Dropout(0.5),
      Dense(1)  # Assuming a regression task for predicting 'totalWords'
  ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    model.summary()
    return model

# Main function to run the script
def main():
    # Load data
    puzzle_data = load_puzzle_data('buggle-training-data/training_data.json')
    print(f"Got {len(puzzle_data)} puzzles.")
    all_features = extract_features(puzzle_data)
    print(f"Loaded {len(all_features)} features.")
    outcomes = np.array([puzzle['outcomes']['totalWords'] for puzzle in puzzle_data])
    print(f"Loaded {len(outcomes)} outcomes.")

    # Preprocess data
    X_train, X_val, X_test, y_train, y_val, y_test = preprocess_data(all_features, outcomes)
    print(f"Preprocessed {len(X_train)} training examples and {len(X_test)} test examples and {len(X_val)} validation examples.")

    model = build_model()

    # Set up early stopping
    early_stopping_monitor = EarlyStopping(
        monitor='val_mean_absolute_error',
        patience=10,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored quantity
    )
    
    # Train the model with the early stopping callback
    history = model.fit(
        X_train, y_train, 
        epochs=100, 
        batch_size=32, 
        validation_split=0.2, 
        verbose=1, 
        callbacks=[early_stopping_monitor]
    )
    
    print("Finished training.")

    # Evaluate the model
    test_results = model.evaluate(X_test, y_test, verbose=1)
    print(f"Test Loss: {test_results[0]}, Test MAE: {test_results[1]}")


    # Plot training history
    plt.figure(figsize=(8, 4))
    plt.plot(history.history['mean_absolute_error'], label='MAE (training data)')
    plt.plot(history.history['val_mean_absolute_error'], label='MAE (validation data)')
    plt.title('MAE for Puzzle Prediction')
    plt.ylabel('MAE value')
    plt.xlabel('No. epoch')
    plt.legend(loc="upper right")
    plt.show()

    # def wrapped_predict(X):
    #     return model.predict(X).flatten()

    # results = permutation_importance(wrapped_predict, X_val, y_val, scoring='neg_mean_absolute_error', n_repeats=10, random_state=42)

    # # Get importance and print
    # importance = results.importances_mean
    # feature_names = ['Feature1', 'Feature2', 'Feature3', ...]  # Update these based on your actual features
    # for i, imp in enumerate(importance):
    #     print(f"{feature_names[i]} Importance: {imp}")

# Run the script
if __name__ == '__main__':
    main()


ModuleNotFoundError: No module named 'tensorflow'